In [ ]:
#Import Libraries
import xml.etree.ElementTree as ET
#from fuzzywuzzy import fuzz
import os
import time
import shutil
import zipfile
from inspect import getmembers, isclass, isfunction
import pandas as pd
import Levenshtein as lev
import itertools

import numpy as np
import openpyxl
import json
from IPython.display import Image
import base64
from IPython.display import display, HTML
import time
import re
import csv

import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,TimeoutException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
import calendar
from datetime import datetime, timedelta
import time
import re
import requests
import json
import pandas as pd
import numpy as np



#import pytesseract
#from PIL import Image

In [15]:
#open Chrome if it is not active
try: 
    driver.current_url
    print('Chrome já conectado')
except:
    #older driver = webdriver.Chrome(ChromeDriverManager().install()) needed to be changed
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    #define the time to wait
    wait=WebDriverWait(driver,10)

In [16]:
#abre o MEC
driver.get('https://emec.mec.gov.br/emec/nova')

In [17]:
#preenche a UF como São Paulo 
webelement = '''//select [(@id="sel_sg_uf")]'''
select = Select(driver.find_element(By.XPATH,webelement))
select.select_by_visible_text("São Paulo")
time.sleep(3)
#clica em pesquisar
webelement = '''//input [(@id="btnPesqAvancada")]'''
driver.find_element(By.XPATH,webelement).click()


In [6]:
#_listaCursos = []

In [18]:
#aguarda a seleção aparecer
waitLoading = True
while waitLoading is True:
    time.sleep(0.5)
    webelement = '//table[@id="tbDataGridNova"]//tr'
    try: 
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH,webelement)))
        #print('visivel')
        waitLoading = False
    except:
        #print('invisivel')
        waitLoading = True

In [19]:
#acessa a tabela com as instituições de ensino
janelas = driver.window_handles
#alterna para outra aba
driver.switch_to.window(janelas[0])
#clica e seleciona para exibir 300
webelement = '''//select [(@id="paginationItemCountItemdiv_listar_consulta_avancada")]'''
select = Select(driver.find_element(By.XPATH,webelement))
select.select_by_visible_text("300")
time.sleep(3)

In [64]:
janelas = driver.window_handles
#alterna para outra aba
driver.switch_to.window(janelas[0])
#continuar a partir do erro
_continuar_de = 280
while _continuar_de < 301:
    try:
        janelas = driver.window_handles
        #alterna para outra aba
        driver.switch_to.window(janelas[0])
        #clica na lupa
        for escola in range(_continuar_de,len(driver.find_elements(By.XPATH,'''//table[@id="tbDataGridNova"]//tr''')[:])-1):
            janelas = driver.window_handles
            #alterna para outra aba
            driver.switch_to.window(janelas[0])
            print(escola)
            driver.find_element(By.XPATH,f'''//table[@id="tbDataGridNova"]//tr[{escola}]/td[9]''').click()
            # Obtém a lista de identificadores de todas as janelas abertas
            janelas = driver.window_handles
            #alterna para outra aba
            driver.switch_to.window(janelas[1])

            # Muda o foco para a nova janela
            for janela in janelas:
                driver.switch_to.window(janela)
            #entra no frame
            driver.switch_to.default_content()
            frame = driver.find_element(By.CSS_SELECTOR,'iframe[name="tabIframe2"]')
            driver.switch_to.frame(frame)
            #aguarda a seleção aparecer
            waitLoading = True
            while waitLoading is True:
                time.sleep(0.5)
                webelement = '//table[@class="avalTabCampos"]//tr//td[contains(text(),"Mantenedora")]'
                try: 
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,webelement)))
                    #print('visivel')
                    waitLoading = False
                except :
                    #print('invisivel')
                    waitLoading = True

            #obtendo dados da instituição
            _mantenedora        = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"Mantenedora")]//parent::tr//td[2]//*''').text
            _cnpj               = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"CNPJ")]//parent::tr//td[2]''').text
            _natJuridica        = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"Natureza Jurídica")]//parent::tr//td[2]''').text
            try:
                _representanteLegal = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"Representante Legal")]//parent::tr//td[2]''').text
            except :
                _representanteLegal = 'Nenhum'
            _nomeIES            = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"Nome da IES - Sigla")]//parent::tr//td[2]//*''').text
            _situacaoIES        = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"Situação")]//parent::tr//td[2]''').text
            _email              = driver.find_element(By.XPATH,'''//table[@class="avalTabCampos"]//tr//td[contains(text(),"mail")]//parent::tr//td[2]''').text


            #aguarda a gradução aparecer
            waitLoading = True
            while waitLoading is True:
                time.sleep(2)
                webelement = '''//li[@id="cursos"]'''
                try: 
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,webelement)))
                    #print('visivel')
                    waitLoading = False
                except :
                    #print('invisivel')
                    waitLoading = True

            #clica em gradução
            webelement = '''//li[@id="cursos"]'''
            time.sleep(1)
            driver.find_element(By.XPATH,webelement).click()

            #entra no frame
            driver.switch_to.default_content()
            frame = driver.find_element(By.CSS_SELECTOR,'iframe[name="tabIframe2"]')
            driver.switch_to.frame(frame)

            #aguarda a seleção aparecer
            waitLoading = True
            while waitLoading is True:
                time.sleep(1)
                webelement = '//select[@id="paginationItemCountItemdivListarCurso"]'
                try: 
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,webelement)))
                    #print('visivel')
                    waitLoading = False
                except :
                    #print('invisivel')
                    waitLoading = True

            #muda a seleção para 1000
            webelement = '''//select [(@id="paginationItemCountItemdivListarCurso")]'''
            select = Select(driver.find_element(By.XPATH,webelement))
            select.select_by_visible_text("1000")
            time.sleep(3)

            #aguarda a seleção aparecer
            waitLoading = True
            while waitLoading is True:
                time.sleep(0.5)
                webelement = '//table[@id="listar-ies-cadastro"]'
                try: 
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,webelement)))
                    #print('visivel')
                    waitLoading = False
                except :
                    #print('invisivel')
                    waitLoading = True


            #mapeia os cursos
            webelement = '''//html/body'''
            driver.find_element(By.XPATH,webelement).send_keys(Keys.PAGE_DOWN)
            for curso in range(1,len(driver.find_elements(By.XPATH,'''//table[@id="listar-ies-cadastro"]/tbody/tr/td[1]/a/div'''))):
                _nomeCurso          = driver.find_element(By.XPATH,f'''//table[@id="listar-ies-cadastro"]/tbody/tr[{curso}]/td[1]/a/div''').text
                #formata para remover espaços duplicados
                _nomeCurso = ' '.join(_nomeCurso.split())

                #armazena na lista
                _listaCursos.append({
                    '_mantenedora':_mantenedora,       
                    '_cnpj':_cnpj,              
                    '_natJuridica':_natJuridica,       
                    '_representanteLegal':_representanteLegal,
                    '_nomeIES':_nomeIES,           
                    '_situacaoIES':_situacaoIES,       
                    '_email':_email,  
                    '_nomeCurso':_nomeCurso
                })
            #fecha a aba
            driver.close()
            #adiciona uma variável
            _continuar_de+=1
            print('sucesso, vou para a página',_continuar_de)
    except KeyboardInterrupt:
        1+1
        print('vou retentar a escola',_continuar_de)

280
sucesso, vou para a página 281
281
sucesso, vou para a página 282
282
sucesso, vou para a página 283
283
sucesso, vou para a página 284
284
sucesso, vou para a página 285
285
sucesso, vou para a página 286
286
sucesso, vou para a página 287
287
sucesso, vou para a página 288
288
sucesso, vou para a página 289
289
sucesso, vou para a página 290
290
sucesso, vou para a página 291
291
sucesso, vou para a página 292
292
sucesso, vou para a página 293
293
sucesso, vou para a página 294
294
sucesso, vou para a página 295
295
sucesso, vou para a página 296
296
sucesso, vou para a página 297
297
sucesso, vou para a página 298
298
sucesso, vou para a página 299
299
sucesso, vou para a página 300
300
sucesso, vou para a página 301


In [66]:
dfMEC = pd.DataFrame(_listaCursos)

In [67]:
dfMEC = dfMEC.drop_duplicates()

In [69]:
dfMEC.to_csv('cursosMEC2.csv',sep=';',encoding='ANSI')